In [44]:
import numpy as np

def softmax_function(a):
    # 全连接层转概率分布
    c = np.max(a)
    expa = np.exp(a - c)
    # overflow avoid
    sum_exp = np.sum(expa)
    y = expa / sum_exp
    return y

def cross_entropy_error(pred,real):
    # 就是重点关注 在正确的上面,是不是熵越小
    # 还是符合 误差越大,这个数字就越大
    # 交叉熵损失函数,用于衡量输入张量和真实张量的差值
    delta = 1e-7 # avoid overflow
    return -np.sum(real * np.log(pred+delta))

def numerical_gradient(func,input_points_array):
    # 计算梯度,就是在输入维度的每个维度上求其导数
    delta = 1e-4
    tmp_shape = input_points_array.shape
    # 缓存原始矩阵形状 如 2,3 ! size 返回的是长度
    # 扁平化处理
    if input_points_array.ndim != 1:
        input_points_array = input_points_array.reshape(1,input_points_array.size).squeeze()
    
    # 扁平化后直接迭代全部,求偏导
    grad = np.zeros_like(input_points_array)
    for index in range(input_points_array.size):
        temp_val = input_points_array[index]
        # f(x+h)
        input_points_array[index] = temp_val + delta
        fv1 = func(input_points_array)

        input_points_array[index] = temp_val - delta
        fv2 = func(input_points_array)

        grad[index] = (fv1 - fv2) / (delta * 2)
        input_points_array[index] = temp_val
    
    # 还原原始维度
    input_points_array = input_points_array.reshape(tmp_shape)
    grad = grad.reshape(tmp_shape)
    return grad

In [45]:
class simpleNet:
    def __init__(self):
        # 定义一个 2x3 的矩阵
        self.W = np.random.randn(2,3)
    def predict(self,x):
        return np.dot(x,self.W)
    def loss(self,input_array,real):
        # real 是one-hot 的概率
        z = self.predict(input_array)
        y = softmax_function(z) # 向量转概率分布
        loss = cross_entropy_error(y,real)
        return loss
network = simpleNet()
input_array = np.array([0.6,0.9])
predict = network.predict(input_array)
print("Predictions :",predict)
np.argmax(predict)
real = np.array([0,0,1])
network.loss(input_array,real)
def f(W):
    return network.loss(input_array,real)
dW = numerical_gradient(f,network.W)
print(dW)

Predictions : [-1.43840304  0.02013181  2.71470971]
[[ 0.0087043   0.0374255  -0.0461298 ]
 [ 0.01305645  0.05613825 -0.06919471]]
